In [87]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsClassifier
import multiprocessing
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_squared_error

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

# Descarga del target
# ==============================================================================
from google.colab import files

#Primer DataFrame

In [88]:
DataFrame2 = pd.read_csv('/content/short_df_Encoded.csv')

In [89]:
DataFrame2.head()

,Unnamed: 0,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0,0.0
1,1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0,1.0
2,2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0,1.0
3,3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0,0.0
4,4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0,0.0


In [90]:
DataFrame2.drop(columns = {'Unnamed: 0'}, inplace = True)

In [91]:
DataFrame2.head()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0,0.0


In [92]:
DataFrame2Train = DataFrame2.loc[:,DataFrame2.columns != 'Decision']

In [93]:
DataFrame2Test  =  DataFrame2.loc[:,'Decision']

In [94]:
DataFrame2Train.shape, DataFrame2Test.shape, DataFrame2.shape

((16772, 9), (16772,), (16772, 10))

##LightGBM


In [95]:
x_train, x_test, y_train, y_test = train_test_split(DataFrame2Train,DataFrame2Test,test_size = 0.2, random_state = 0)

In [96]:
model = LGBMRegressor(boosting_type = "goss", n_estimators = 1100, learning_rate = 0.008, random_state = 123)

In [97]:
model.fit(x_train, y_train)

LGBMRegressor(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.008, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [98]:
preds = model.predict(x_test)

In [99]:
loss = log_loss(y_test, preds)
loss

0.4909067289883312

In [100]:
#val = model.feature_importances_

In [101]:
#df = pd.DataFrame(val)

In [102]:
#df.columns = ["Value"]
#df.sort_values(by = "Value").plot.bar()

##Catboost

In [103]:
!pip install catboost

In [104]:
from catboost import CatBoostRegressor

In [105]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(DataFrame2Train,DataFrame2Test,test_size = 0.2, random_state = 0)

In [106]:
modelCatReg1 = CatBoostRegressor(num_boost_round = 1500, learning_rate = 0.007)

#Entreno al modelo
modelCatReg1.fit(x_train2, y_train2)
#Predicciones
preds_CatReg = modelCatReg1.predict(x_test2)
print("CatReg = ", preds_CatReg)

0:	learn: 0.4939131	total: 4.06ms	remaining: 6.08s
1:	learn: 0.4927485	total: 6.87ms	remaining: 5.14s
2:	learn: 0.4915755	total: 9.68ms	remaining: 4.83s
3:	learn: 0.4904074	total: 12.5ms	remaining: 4.66s
4:	learn: 0.4892143	total: 15.2ms	remaining: 4.55s
5:	learn: 0.4880268	total: 18ms	remaining: 4.49s
6:	learn: 0.4868536	total: 20.8ms	remaining: 4.43s
7:	learn: 0.4857370	total: 23.7ms	remaining: 4.41s
8:	learn: 0.4846057	total: 26.6ms	remaining: 4.41s
9:	learn: 0.4834679	total: 29.4ms	remaining: 4.38s
10:	learn: 0.4823718	total: 31.9ms	remaining: 4.32s
11:	learn: 0.4812722	total: 34.6ms	remaining: 4.29s
12:	learn: 0.4801667	total: 37.3ms	remaining: 4.27s
13:	learn: 0.4790675	total: 40ms	remaining: 4.25s
14:	learn: 0.4779984	total: 42.8ms	remaining: 4.24s
15:	learn: 0.4769706	total: 45.8ms	remaining: 4.25s
16:	learn: 0.4759180	total: 48.5ms	remaining: 4.23s
17:	learn: 0.4748625	total: 51.2ms	remaining: 4.22s
18:	learn: 0.4738212	total: 54.2ms	remaining: 4.22s
19:	learn: 0.4728012	total

In [107]:
#Error del modelo
rmse1 = np.sqrt(mean_squared_error(y_test2, preds_CatReg))
print("RMSE: %f" % (rmse1))

RMSE: 0.375398


In [108]:
accuracy1 = accuracy_score(y_test2, preds_CatReg.round().astype('int'))
print("Accuracy: %.2f%%" % (accuracy1 * 100.0))

Accuracy: 79.73%


In [109]:
lossCat = log_loss(y_test2, preds_CatReg)
lossCat

0.43018030392205603

##Xgboost

In [110]:
import xgboost as xgb

In [111]:
x_train5, x_test5, y_train5, y_test5 = train_test_split(DataFrame2Train,DataFrame2Test,test_size = 0.2, random_state = 0)

In [112]:
xg_Regressor1 = xgb.XGBRegressor(objective ='reg:squarederror',colsample_bytree= 0.8, gamma= 1.5, learning_rate= 0.01, max_depth= 20, min_child_weight= 1, n_estimators= 200, subsample= 0.8)

In [113]:
xg_Regressor1.fit(x_train5,y_train5)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=1.5,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [114]:
predsRegressor1 = xg_Regressor1.predict(x_test5)

In [115]:
value_pred_Regressor1 = pd.DataFrame()
value_pred_Regressor1['pred'] = predsRegressor1
value_pred_Regressor1['pred'].value_counts()

0.880869    117
0.905074     67
0.895266     61
0.148149     42
0.880080     42
           ... 
0.144152      1
0.627397      1
0.584428      1
0.526376      1
0.342099      1
Name: pred, Length: 1931, dtype: int64

In [116]:
rmseRegressor1 = np.sqrt(mean_squared_error(y_test5, predsRegressor1))
print("RMSE: %f" % (rmseRegressor1))

RMSE: 0.371824


In [117]:
lossRegressor1 = log_loss(y_test5, predsRegressor1)
print(f"Log Loss is {lossRegressor1}")

Log Loss is 0.4387157682046272


#Segundo DataFrame


In [118]:
DataFrame3 = pd.read_csv('/content/DataFrame_train_encoded1.csv')

In [119]:
DataFrame3.head()

,Unnamed: 0,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,0,15.635076,2016,1,1,0,0,1,0,0,0,1,0,0,0
1,1,10.940740,2016,0,0,1,1,0,0,0,0,1,0,0,0
2,2,11.336971,2016,0,0,1,1,0,0,0,0,0,1,0,0
3,3,15.819943,2018,0,1,0,1,0,0,0,0,0,1,0,0
4,4,16.407566,2018,0,1,0,1,0,0,0,0,0,1,0,0


In [120]:
DataFrame3.drop(columns = {'Unnamed: 0'}, inplace = True)

In [121]:
DataFrame3.head()

,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,15.635076,2016,1,1,0,0,1,0,0,0,1,0,0,0
1,10.940740,2016,0,0,1,1,0,0,0,0,1,0,0,0
2,11.336971,2016,0,0,1,1,0,0,0,0,0,1,0,0
3,15.819943,2018,0,1,0,1,0,0,0,0,0,1,0,0
4,16.407566,2018,0,1,0,1,0,0,0,0,0,1,0,0


In [122]:
DataFrame3Train = DataFrame3.loc[:,DataFrame3.columns != 'Decision']

In [123]:
DataFrame3Test  =  DataFrame3.loc[:,'Decision']

In [124]:
DataFrame3Train.shape, DataFrame3Test.shape, DataFrame3.shape

((16772, 13), (16772,), (16772, 14))

##LightGBM

In [125]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(DataFrame3Train,DataFrame3Test,test_size = 0.2, random_state = 0)

In [126]:
modelLGBMReg2 = LGBMRegressor(boosting_type = "goss", n_estimators = 200, learning_rate = 0.1, random_state = 123)

In [127]:
modelLGBMReg2.fit(x_train3, y_train3)

LGBMRegressor(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
              random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [128]:
predsLGBMReg2 = modelLGBMReg2.predict(x_test3)

In [129]:
loss = log_loss(y_test3, predsLGBMReg2)
loss

0.6868113094673227

##Catboost

In [130]:
x_train4, x_test4, y_train4, y_test4 = train_test_split(DataFrame3Train,DataFrame3Test,test_size = 0.2, random_state = 0)

In [131]:
modelCatReg2 = CatBoostRegressor(num_boost_round = 1500, learning_rate = 0.007)

#Entreno al modelo
modelCatReg2.fit(x_train4, y_train4)
#Predicciones
preds_CatReg2 = modelCatReg2.predict(x_test4)
print("CatReg = ", preds_CatReg2)

0:	learn: 0.4938924	total: 3.67ms	remaining: 5.5s
1:	learn: 0.4926584	total: 6.67ms	remaining: 4.99s
2:	learn: 0.4914571	total: 9.62ms	remaining: 4.8s
3:	learn: 0.4902204	total: 12.5ms	remaining: 4.67s
4:	learn: 0.4889856	total: 15.3ms	remaining: 4.58s
5:	learn: 0.4878093	total: 18.2ms	remaining: 4.54s
6:	learn: 0.4866340	total: 22.6ms	remaining: 4.82s
7:	learn: 0.4854665	total: 25.5ms	remaining: 4.76s
8:	learn: 0.4842984	total: 28.4ms	remaining: 4.7s
9:	learn: 0.4831641	total: 31.2ms	remaining: 4.64s
10:	learn: 0.4820080	total: 33.9ms	remaining: 4.59s
11:	learn: 0.4809635	total: 36.2ms	remaining: 4.49s
12:	learn: 0.4799029	total: 39ms	remaining: 4.46s
13:	learn: 0.4787655	total: 41.7ms	remaining: 4.43s
14:	learn: 0.4777009	total: 44.5ms	remaining: 4.41s
15:	learn: 0.4766199	total: 47.3ms	remaining: 4.39s
16:	learn: 0.4755396	total: 50.2ms	remaining: 4.38s
17:	learn: 0.4744715	total: 52.9ms	remaining: 4.36s
18:	learn: 0.4734059	total: 55.7ms	remaining: 4.34s
19:	learn: 0.4724277	total:

In [132]:
#Error del modelo
rmse2 = np.sqrt(mean_squared_error(y_test4, preds_CatReg2))
print("RMSE: %f" % (rmse2))

RMSE: 0.361428


In [133]:
accuracy2 = accuracy_score(y_test4, preds_CatReg2.round().astype('int'))
print("Accuracy: %.2f%%" % (accuracy2 * 100.0))

Accuracy: 80.36%


In [134]:
lossCat2 = log_loss(y_test4, preds_CatReg2)
lossCat2

0.4010891877647015

###Test

In [135]:
DataFrameTest3 = pd.read_csv('/content/DataFrame_test_encoded1.csv')

In [136]:
DataFrameTest3.head()

,Unnamed: 0,Opportunity_ID,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,2,10689,11.905436,2019,1,1,0,1,0,0,0,1,0,0,0
1,7,10690,11.787392,2019,1,1,0,1,0,0,0,1,0,0,0
2,8,10691,9.954062,2019,1,1,0,0,0,1,0,0,1,0,0
3,14,10692,12.338534,2019,1,1,0,0,0,1,0,0,1,0,0
4,15,10693,8.657390,2019,0,0,0,1,0,0,0,0,1,0,0


In [137]:
DataFrameTest3.drop(columns = {'Unnamed: 0'}, inplace = True)

In [138]:
DataFrameTest3.head()

,Opportunity_ID,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,10689,11.905436,2019,1,1,0,1,0,0,0,1,0,0,0
1,10690,11.787392,2019,1,1,0,1,0,0,0,1,0,0,0
2,10691,9.954062,2019,1,1,0,0,0,1,0,0,1,0,0
3,10692,12.338534,2019,1,1,0,0,0,1,0,0,1,0,0
4,10693,8.657390,2019,0,0,0,1,0,0,0,0,1,0,0


In [139]:
DescargaPredCatRegressor2 = pd.DataFrame()
DescargaPredCatRegressor2['Opportunity_ID'] = DataFrameTest3['Opportunity_ID']
DescargaPredCatRegressor2.set_index('Opportunity_ID', inplace = True)

In [140]:
DataFrameTest3 = DataFrameTest3.drop(columns={'Opportunity_ID'})
DataFrameTest3.head()

,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,11.905436,2019,1,1,0,1,0,0,0,1,0,0,0
1,11.787392,2019,1,1,0,1,0,0,0,1,0,0,0
2,9.954062,2019,1,1,0,0,0,1,0,0,1,0,0
3,12.338534,2019,1,1,0,0,0,1,0,0,1,0,0
4,8.657390,2019,0,0,0,1,0,0,0,0,1,0,0


In [141]:
predCatRegressor_Test2 = modelCatReg2.predict(DataFrameTest3)

In [142]:
DescargaPredCatRegressor2['target'] = predCatRegressor_Test2
DescargaPredCatRegressor2

,target
Opportunity_ID,
10689,0.304905
10690,0.324751
10691,0.566514
10692,0.587634
10693,0.518918
...,...
12364,0.345521
12365,0.565242
12366,0.631785


In [143]:
DescargaPredCatRegressor2['target'].value_counts()

0.231820    11
0.525371    10
0.578479    10
0.564069    10
0.585844     9
            ..
0.501878     1
0.266543     1
0.236663     1
0.251186     1
0.118276     1
Name: target, Length: 1084, dtype: int64

In [144]:
csv4 = DescargaPredCatRegressor2.to_csv('catboost_regression_pred2.csv')
files.download('catboost_regression_pred2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Xgboost

In [145]:
x_train6, x_test6, y_train6, y_test6 = train_test_split(DataFrame3Train,DataFrame3Test,test_size = 0.2, random_state = 0)

In [146]:
xg_Regressor2 = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree= 0.8, gamma= 1.5, learning_rate= 0.01, max_depth= 20, min_child_weight= 1, n_estimators= 250, subsample= 0.8)

In [147]:
xg_Regressor2.fit(x_train6,y_train6)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=1.5,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=250,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [148]:
predsRegressor2 = xg_Regressor2.predict(x_test6)

In [149]:
value_pred_Regressor2 = pd.DataFrame()
value_pred_Regressor2['pred'] = predsRegressor2
value_pred_Regressor2['pred'].value_counts()

0.058635    62
0.982250    29
0.939142    26
0.085557    21
0.937671    19
            ..
0.161760     1
0.818917     1
0.686106     1
0.723447     1
0.104004     1
Name: pred, Length: 2469, dtype: int64

In [150]:
rmseRegressor2 = np.sqrt(mean_squared_error(y_test6, predsRegressor2))
print("RMSE: %f" % (rmseRegressor2))

RMSE: 0.357799


In [151]:
lossRegressor2 = log_loss(y_test6, predsRegressor2)
print(f"Log Loss is {lossRegressor2}")

Log Loss is 0.4071269649012922


##VotingRegressor

In [152]:
from sklearn.ensemble import VotingRegressor

In [153]:
reg1 = xg_Regressor2

In [154]:
reg2 = modelCatReg2

In [155]:
vot_reg = VotingRegressor([('xg',reg1),('cat',reg2)])

In [156]:
vot_reg.fit(x_train6,y_train6)

0:	learn: 0.4938924	total: 3.4ms	remaining: 5.11s
1:	learn: 0.4926584	total: 6.39ms	remaining: 4.79s
2:	learn: 0.4914571	total: 9.22ms	remaining: 4.6s
3:	learn: 0.4902204	total: 12ms	remaining: 4.48s
4:	learn: 0.4889856	total: 14.7ms	remaining: 4.4s
5:	learn: 0.4878093	total: 17.6ms	remaining: 4.37s
6:	learn: 0.4866340	total: 20.3ms	remaining: 4.33s
7:	learn: 0.4854665	total: 23.1ms	remaining: 4.3s
8:	learn: 0.4842984	total: 25.9ms	remaining: 4.28s
9:	learn: 0.4831641	total: 28.6ms	remaining: 4.26s
10:	learn: 0.4820080	total: 31.3ms	remaining: 4.23s
11:	learn: 0.4809635	total: 33.6ms	remaining: 4.16s
12:	learn: 0.4799029	total: 36.5ms	remaining: 4.17s
13:	learn: 0.4787655	total: 39.3ms	remaining: 4.17s
14:	learn: 0.4777009	total: 42.1ms	remaining: 4.16s
15:	learn: 0.4766199	total: 44.8ms	remaining: 4.16s
16:	learn: 0.4755396	total: 47.6ms	remaining: 4.15s
17:	learn: 0.4744715	total: 50.3ms	remaining: 4.14s
18:	learn: 0.4734059	total: 53ms	remaining: 4.13s
19:	learn: 0.4724277	total: 56

VotingRegressor(estimators=[('xg',
                             XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.8, gamma=1.5,
                                          importance_type='gain',
                                          learning_rate=0.01, max_delta_step=0,
                                          max_depth=20, min_child_weight=1,
                                          missing=None, n_estimators=250,
                                          n_jobs=1, nthread=None,
                                          objective='reg:squarederror',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=0.8,
                           

In [157]:
preds_vot = vot_reg.predict(x_test6)

In [158]:
value_pred_vot_Regressor1 = pd.DataFrame()
value_pred_vot_Regressor1['pred'] = preds_vot
value_pred_vot_Regressor1['pred'].value_counts()

0.032743    28
0.032763    24
0.954046    18
0.953027    17
0.953514    15
            ..
0.554907     1
0.052345     1
0.239572     1
0.451778     1
0.685134     1
Name: pred, Length: 2610, dtype: int64

In [159]:
rmseRegressorvot1 = np.sqrt(mean_squared_error(y_test6, preds_vot))
print("RMSE: %f" % (rmseRegressorvot1))

RMSE: 0.357751


In [160]:
lossRegressorvot1 = log_loss(y_test6, preds_vot)
print(f"Log Loss is {lossRegressorvot1}")

Log Loss is 0.40114264718717874


#Tercer DataFrame

In [161]:
df = pd.read_csv( "/content/Training.csv" )

In [162]:
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df['Total_Amount']*0.0096
df.loc[df['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df['Total_Amount']*1.17
df.loc[df['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = df['Total_Amount']*0.70
df.loc[df['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df['Total_Amount']*1.29
df.loc[df['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'
df.dropna(inplace = True)
df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'],errors='coerce')
df["Year Created"] = df["Opportunity_Created_Date"].dt.year
df["Month Created"] =df["Opportunity_Created_Date"].dt.month
df["Expensive"] = np.where(df["Total_Taxable_Amount"] >= 80000, 1, 0)
df['Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'],errors='coerce')
df['Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'],errors='coerce')
df = df[df["Total_Amount"] > 0]
df["Total_Amount"] = np.log(df["Total_Amount"])
entrenamiento_by_region = df.groupby("Region").agg({"Stage" : "count"})
entrenamiento_by_region.rename(columns = {"Stage" : "Amount of Cases"}, inplace = True)
entrenamiento_won_by_region = df[df["Stage"] == "Closed Won"].groupby("Region").agg({"Stage" : "count"})
entrenamiento_won_by_region.rename(columns = {"Stage" : "Cases won"}, inplace = True)

entrenamiento_won_by_region["percentage"] = (entrenamiento_won_by_region["Cases won"] / entrenamiento_by_region["Amount of Cases"]) * 100
df = df.merge(entrenamiento_won_by_region, on = "Region")
df["Años en entregar"] = df["Delivery_Year"] - df["Year Created"]

In [163]:
short_df = df[['Opportunity_ID', "Total_Amount", "Años en entregar","Month Created", "Product_Name", "Opportunity_Owner", 'Stage' ]].rename(columns={'Stage': 'Decision'})
short_df = short_df[ (short_df['Decision'] == 'Closed Won') | (short_df['Decision'] == 'Closed Lost') ]
short_df['Decision'] = np.where(short_df['Decision'] == 'Closed Won',1,0)
short_df.set_index('Opportunity_ID', inplace = True)


short_df.describe()
short_df.dropna(inplace = True)

In [164]:
short_df["Product_Name"] = np.where(short_df.groupby('Product_Name')["Product_Name"].transform(len) > 5, short_df["Product_Name"], "Other")
short_df["Product_Name"].value_counts()

Product_Name_100    968
Product_Name_226    835
Product_Name_231    574
Product_Name_192    437
Other               397
                   ... 
Product_Name_180      6
Product_Name_264      6
Product_Name_391      6
Product_Name_29       6
Product_Name_160      6
Name: Product_Name, Length: 237, dtype: int64

In [165]:
short_df["Opportunity_Owner"] = np.where(short_df.groupby('Opportunity_Owner')["Opportunity_Owner"].transform(len) > 5, short_df["Opportunity_Owner"], "Other")
short_df["Opportunity_Owner"].value_counts()

Person_Name_8     1352
Person_Name_13    1252
Person_Name_50    1071
Person_Name_18     979
Person_Name_43     831
Person_Name_3      760
Person_Name_4      682
Person_Name_32     655
Person_Name_19     614
Person_Name_16     438
Person_Name_23     422
Person_Name_64     408
Person_Name_65     322
Person_Name_49     285
Person_Name_46     238
Person_Name_44     225
Person_Name_66     218
Person_Name_39     210
Person_Name_60     206
Person_Name_48     198
Person_Name_38     193
Person_Name_62     174
Person_Name_52     173
Person_Name_54     168
Person_Name_5      152
Person_Name_59     134
Person_Name_2      118
Person_Name_45     112
Person_Name_41     112
Person_Name_51     111
Person_Name_33      86
Person_Name_14      82
Person_Name_55      79
Person_Name_11      70
Person_Name_30      64
Person_Name_63      61
Person_Name_40      50
Person_Name_25      48
Person_Name_20      48
Person_Name_22      33
Person_Name_26      33
Person_Name_37      21
Person_Name_42      19
Person_Name

In [166]:
mean_encoded_owner = short_df.groupby("Opportunity_Owner")["Decision"].mean().to_dict()
short_df["Opportunity_Owner"] = short_df["Opportunity_Owner"].map(mean_encoded_owner)

In [167]:
mean_encoded_product = short_df.groupby("Product_Name")["Decision"].mean().to_dict()

In [168]:
short_df["Product_Name"] = short_df["Product_Name"].map(mean_encoded_product)

In [169]:
short_df.head()

,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Decision
Opportunity_ID,,,,,,
0,15.635076,1,12,0.186992,0.560776,0
1,10.940740,1,12,0.592975,0.750000,1
19,12.624960,1,12,0.561404,0.261905,0
20,11.702880,1,12,0.592975,0.664537,1
23,12.433069,1,12,0.592975,0.560776,1


In [170]:
X_train7, X_test7, y_train7, y_test7 = train_test_split(
                                        short_df.drop(columns = 'Decision'),
                                        short_df['Decision'],
                                        random_state = 123
                                    )

##Catboost

In [171]:
modelCatReg3 = CatBoostRegressor(learning_rate = 0.0075, verbose = True, random_seed = 123, loss_function = "RMSE", num_boost_round = 1500, max_depth = 16)

In [172]:
#modelCatReg3.fit(X_train7, y_train7)

In [173]:
#preds_CatReg3 = modelCatReg3.predict(X_test7)

In [174]:
#Error del modelo
#rmsecat3 = np.sqrt(mean_squared_error(y_test7, preds_CatReg3))
#print("RMSE: %f" % (rmsecat3))

In [175]:
#losscat3 = log_loss(y_test7, preds_CatReg3)
#losscat3

##Xgboost

In [176]:
xg_Regressor3 = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree= 0.8, gamma= 1.5, learning_rate= 0.01, max_depth= 20, min_child_weight= 1, n_estimators= 250, subsample= 0.8)

In [177]:
#xg_Regressor3.fit(X_train7, y_train7)

In [178]:
#predsRegressor3 = xg_Regressor3.predict(X_test7)

In [179]:
#value_pred_Regressor3 = pd.DataFrame()
#value_pred_Regressor3['pred'] = predsRegressor3
#value_pred_Regressor3['pred'].value_counts()

In [180]:
#rmseRegressor3 = np.sqrt(mean_squared_error(y_test7, predsRegressor3))
#print("RMSE: %f" % (rmseRegressor3))

In [181]:
#lossRegressor3 = log_loss(y_test7, predsRegressor3)
#print(f"Log Loss is {lossRegressor3}")

SyntaxError: ignored

##VotingRegressor

In [182]:
reg3 = xg_Regressor3

In [183]:
reg4 = modelCatReg3

In [184]:
vot_reg2 = VotingRegressor([('xg',reg3),('cat',reg4)])

In [185]:
vot_reg2.fit(X_train7,y_train7)

0:	learn: 0.4957081	total: 733ms	remaining: 18m 18s
1:	learn: 0.4945013	total: 740ms	remaining: 9m 14s
2:	learn: 0.4931393	total: 1.26s	remaining: 10m 28s
3:	learn: 0.4918283	total: 1.32s	remaining: 8m 14s
4:	learn: 0.4904242	total: 1.86s	remaining: 9m 16s
5:	learn: 0.4890736	total: 2.38s	remaining: 9m 54s
6:	learn: 0.4878103	total: 2.44s	remaining: 8m 41s
7:	learn: 0.4864851	total: 2.98s	remaining: 9m 15s
8:	learn: 0.4851714	total: 3.51s	remaining: 9m 41s
9:	learn: 0.4839008	total: 4.04s	remaining: 10m 2s
10:	learn: 0.4826141	total: 4.57s	remaining: 10m 18s
11:	learn: 0.4813590	total: 5.1s	remaining: 10m 32s
12:	learn: 0.4801196	total: 5.63s	remaining: 10m 44s
13:	learn: 0.4789596	total: 5.69s	remaining: 10m 3s
14:	learn: 0.4777761	total: 6.22s	remaining: 10m 16s
15:	learn: 0.4765658	total: 6.75s	remaining: 10m 26s
16:	learn: 0.4753781	total: 7.27s	remaining: 10m 34s
17:	learn: 0.4742249	total: 7.33s	remaining: 10m 3s
18:	learn: 0.4730475	total: 7.86s	remaining: 10m 12s
19:	learn: 0.4

VotingRegressor(estimators=[('xg',
                             XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.8, gamma=1.5,
                                          importance_type='gain',
                                          learning_rate=0.01, max_delta_step=0,
                                          max_depth=20, min_child_weight=1,
                                          missing=None, n_estimators=250,
                                          n_jobs=1, nthread=None,
                                          objective='reg:squarederror',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=0.8,
                           

In [186]:
preds_vot2 = vot_reg2.predict(X_test7)

In [187]:
value_pred_vot_Regressor2 = pd.DataFrame()
value_pred_vot_Regressor2['pred'] = preds_vot2
value_pred_vot_Regressor2['pred'].value_counts()

0.071466    11
0.038815    10
0.864906     9
0.120166     8
0.607750     7
            ..
0.297938     1
0.672863     1
0.265898     1
0.209129     1
0.820463     1
Name: pred, Length: 3075, dtype: int64

In [188]:
rmseRegressorvot2 = np.sqrt(mean_squared_error(y_test7, preds_vot2))
print("RMSE: %f" % (rmseRegressorvot2))

RMSE: 0.366192


In [189]:
lossRegressorvot2 = log_loss(y_test7, preds_vot2)
print(f"Log Loss is {lossRegressorvot2}")

Log Loss is 0.41908276402757594


###Test

In [214]:
df_test = pd.read_csv( "/content/Test.csv" )
df_test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,Source_3,South Africa,Account_Name_1686,Opportunity_Name_11120,12364,6508,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_1,Non Binding,Delivery_Terms_7,4/26/2019,None,None,None,None,None,None,NaT,5/24/2019,4/26/2019,Person_Name_13,Product_Family_213,Product_Name_447,USD,0.300,USD,0.30000,5/1/2019,5/10/2019,2019 - 5,Q2,2019,NaT,0,USD,124740.0,USD,147750.0,Prod_Category_A_None
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,None,United States,Account_Name_768,Opportunity_Name_9694,12365,6510,Person_Name_38,Person_Name_38,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_4,4/26/2019,None,None,None,None,None,None,N

In [215]:
df_test.loc[df_test['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df_test['Total_Amount']*0.0096
df_test.loc[df_test['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df_test.loc[df_test['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df_test['Total_Amount']*1.17
df_test.loc[df_test['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] 
df_test.loc[df_test['Total_Amount_Currency'] == 'AUD', 'Total_Taxable_Amount'] = df_test['Total_Amount']*0.70
df_test.loc[df_test['Total_Amount_Currency'] == 'AUD', 'Total_Taxable_Amount_Currency'] = 'USD'

df_test.loc[df_test['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df_test['Total_Amount']*1.29
df_test.loc[df_test['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

df_test['Opportunity_Created_Date'] = pd.to_datetime(df_test['Opportunity_Created_Date'],errors='coerce')

df_test["Year Created"] = df_test["Opportunity_Created_Date"].dt.year
df_test["Month Created"] =df_test["Opportunity_Created_Date"].dt.month
df_test["Expensive"] = np.where(df_test["Total_Amount"] >= 80000, 1, 0)


df_test = df_test.merge(entrenamiento_won_by_region, on = "Region")
df_test["Años en entregar"] = df_test["Delivery_Year"] - df_test["Year Created"]
df_test["Total_Taxable_Amount"].describe()

count    2.551000e+03
mean     3.393353e+06
std      2.430590e+07
min      0.000000e+00
25%      6.987750e+04
50%      2.195684e+05
75%      7.818624e+05
max      6.808725e+08
Name: Total_Taxable_Amount, dtype: float64

In [216]:
df_test.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Year Created,Month Created,Expensive,Cases won,percentage,Años en entregar
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,189820.80,EUR,367419.0,Prod_Category_A_None,2019,4,1,2467,53.179565,0
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,91990.08,EUR,367419.0,Prod_Category_A_None,2019,4,1,2467,53.179565,0
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,148069.35,EUR,367419.0,Prod_Category_A_None,2019,4,1,2467,53.179565,0
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,284731.20,EUR,757783.5,Prod_Category_A_None,2019,4,1,2467,53.179565,0
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,183980.16,EUR,757783.5,Prod_Category_A_None,2019,4,1,2467,53.179565,0


In [217]:
df_test["Total_Amount"] = np.log(df_test["Total_Amount"])
df_test = df_test[['Opportunity_ID', "Total_Amount", "Años en entregar","Month Created", "Product_Name", "Opportunity_Owner"]]
df_test = df_test.drop_duplicates('Opportunity_ID',keep = 'first')
df_test.set_index('Opportunity_ID', inplace = True)
df_test["Product_Name"] = df_test["Product_Name"].map(mean_encoded_product)
df_test["Opportunity_Owner"] = df_test["Opportunity_Owner"].map(mean_encoded_owner)
#df_test["Source "] = df_test["Source "].map(mean_encoded_source)

In [218]:
df_test["Product_Name"].fillna(mean_encoded_product["Other"], inplace = True)
df_test["Opportunity_Owner"].fillna(mean_encoded_owner["Other"], inplace = True)

In [219]:
predvot2_test = vot_reg2.predict(df_test)

In [220]:
subirvot = pd.DataFrame()
subirvot['Opportunity_ID'] = df_test.reset_index()['Opportunity_ID']
subirvot['target'] = predvot2_test
subirvot.sort_values('Opportunity_ID',inplace = True)
subirvot = subirvot.drop_duplicates('Opportunity_ID',keep = 'last')
subirvot.set_index('Opportunity_ID', inplace = True)
subirvot

,target
Opportunity_ID,
10689,0.417648
10690,0.499192
10691,0.463150
10692,0.389077
10693,0.882226
...,...
12364,0.944007
12365,0.304672
12366,0.183927


In [221]:
subirvot.describe()

,target
count,1567.000000
mean,0.518232
std,0.257251
min,0.012675
25%,0.293967
50%,0.512216
75%,0.751314
max,0.977129


In [222]:
csv8 = subirvot.to_csv('vot_pred2.csv')
files.download('vot_pred2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>